In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import optuna
from darts.timeseries import TimeSeries
from darts.models import FourTheta
from darts.metrics import mape,mae
import pickle
import os

In [46]:
from darts.metrics import mape,mae,mse
from darts.timeseries import TimeSeries
#from darts.models import Prophet,CatBoostModel,FFT
from darts.utils.statistics import check_seasonality, plot_acf, plot_residuals_analysis, plot_hist,extract_trend_and_seasonality
from darts.utils.utils import SeasonalityMode
import logging
from darts.utils.utils import ModelMode
from darts.utils.utils import TrendMode
from darts.models import FourTheta, Prophet
from darts.dataprocessing.transformers import Scaler
from darts.dataprocessing.transformers import BoxCox

In [47]:
with open("data/srag_series.pkl", "rb") as f:
    srag_series = pickle.load(f)

def train_test_splits(series, train_proportion_list):
    '''
    Funcao para dividir a serie temporal em treino e teste
    
    Parametros:
    
    *series: serie que sera particionada em treino e validacao.
    *train_proportion_list: Lista que define a proporcao do conjunto
    de treinamento. Ex: series_list = [0.8,0.5] significa primeiro conjunto 
    de treino com 80% dos dados totais e o segundo com 50% dos dados totais.
    '''
    partitions_dict = {'train': [], 'test': []}

    for i, train_proportion in enumerate(train_proportion_list):
        train_size = int(len(series) * train_proportion)
        train, test = series.split_before(train_size)
        partitions_dict['train'].append(train)
        partitions_dict['test'].append(test)

        print(f'Partition {i+1} - Train size: {len(train)}, Test size: {len(test)}, Total size: {len(train)+len(test)}')

    return partitions_dict

In [48]:
train_test_split_SRAG = train_test_splits(srag_series.univariate_component(0),[0.8,0.85])

Partition 1 - Train size: 249, Test size: 63, Total size: 312
Partition 2 - Train size: 265, Test size: 47, Total size: 312


In [77]:
train = train_test_split_SRAG['train'][1].slice(pd.Timestamp("2021-01-01"), pd.Timestamp("2024-12-01"))
val = train_test_split_SRAG['test'][1]

In [ ]:
figure = plt.figure(figsize=(15, 5))
train_test_split_SRAG['train'][0].slice(pd.Timestamp("2022-01-01"), pd.Timestamp("2024-12-01")).plot(label = 'Conjunto de Treino')
train_test_split_SRAG['test'][0].plot(label = 'Conjunto de Teste')
plt.xticks(fontsize = 10)
plt.yticks(fontsize = 15)
plt.xlabel('Tempo', fontsize = 15)
plt.ylabel('Casos', fontsize = 15)
plt.legend()
plt.show()

def objective(trial):
    global model

    params = {
        "theta": trial.suggest_int("theta", 0, 5),
        "seasonality_period": trial.suggest_int("seasonality_period", 15,80),
        "season_mode": trial.suggest_categorical("season_mode", [
            SeasonalityMode.ADDITIVE,
            SeasonalityMode.MULTIPLICATIVE
        ]),
        "model_mode": trial.suggest_categorical("model_mode", [
            ModelMode.ADDITIVE,
            ModelMode.MULTIPLICATIVE
        ]),
        "trend_mode": trial.suggest_categorical("trend_mode", [
            TrendMode.LINEAR,
            TrendMode.EXPONENTIAL
        ])
    }

    model = FourTheta(**params)
    model.fit(train)
    predictions = model.predict(len(val))

    return mape(val, predictions)

In [ ]:
def callback(study,trial):
    global best_model
    if study.best_trial == trial:
        best_model = model

study = optuna.create_study(direction = "minimize")
study.optimize(objective, n_trials = 500, callbacks = [callback])

In [1]:
preds = best_model.predict(n=len(val)+50)
mape_pred = mape(val,preds)
fig = plt.figure(figsize=(15,5))
srag_series.univariate_component(0).slice(pd.Timestamp("2020-01-01"), pd.Timestamp("2025-12-01")).plot(label='Série original')
train_test_split_SRAG['train'][1].slice(pd.Timestamp("2022-01-01"), pd.Timestamp("2025-12-01")).plot(label='Série original')
preds.plot(label='Previsão')
plt.plot([],[],' ',label=f'MAPE: {round(mape_pred,2)}')
plt.legend()
plt.show()

NameError: name 'best_model' is not defined